# Радиолокационные данные
***

Для упрощения работы и анализа радиолокационных данных существует множество открытых библиотек на github. Однако есть наиболее оконченная такая как ```openradar```. Вся дальнейшая работа основана на наработках данного репозитория.
```python 
!pip install openradar # Библиотека для работы с радаром
import mmwave as mm    # импорт библиотеки для работы с радаром
```
Ссылка на github репозиторий: [OpenRadar on github](https://github.com/PreSenseRadar/OpenRadar)  
Ссылка на докумкентацию модуля: [Welcome to the mmwave docs!](https://openradar.readthedocs.io/en/latest/)

## Как радар считывает данные

Сначала радар посылает известные ЛЧМ сигналы с использованием одного или нескольких передатчиков. Затем он ожидает, пока эти сигналы не будут прерваны объектами на пути и не отразятся обратно. Дальше радар использует свой приемник или приемники для считывания сигналов в виде аналоговых значений и оцифровки их в типы данных, с которыми мы можем работать (например, int16) с помощью АЦП.

## Упорядочивание данных

Выборка АЦП в данном контексте представляет собой буквальное преобразование принятой электромагнитной волны в цифровой сигнал.

Выборки АЦП хранятся в последовательностях, которые сопоставляются с одним ЛЧМ сигналом. Например, после отправки сигнала, радар может быть сконфигурирован для "тестирования" принятого сигнала 256 раз в течение заданного промежутка времени с равными интервалами выборки.

Один кадр содержит информацию, полученную путем излучения радаром группы из нескольких сигналов. По сути, после выборки одного ЛЧМ мы можем выполнить выборку дополнительных ЛЧМ и сложить результат столько раз, сколько захотим.

Поскольку теперь у вас есть базовое представление о данных, с которыми мы будем работать, теперь мы можем перейти к их использованию. В следующих нескольких модулях "основы" мы поговорим о том, как извлечь важную информацию из этих данных.